In [1]:
import pandas as pd
import json
import Tool
import os
import config
import numpy  as np
import operators_v4
import tqdm
from joblib import Parallel, delayed
import sys
sys.path.append("C:/Users/User/我的雲端硬碟 (owen.lin@mutual-boost.com)/MBQ_Tej_v2/MBQ_tej_v2")
import MBQ_tej_v2_manager
import matplotlib.pyplot as plt
args = json.load(open("./config.json", encoding="utf-8"))
save_path = r'C:\Users\User\Desktop\pc\stcok\Results\Results.xlsx'

In [2]:
Handler_cache_path = args['Handler_cache_path']
print(f"Load Data : {Handler_cache_path}")
CacheHandler = Tool.Handler(path = Handler_cache_path,data_type = args['data_type'])

Load Data : C:\Users\User\Documents\MBQ_TW_DB


In [3]:
# import 更新因子资料
handler = MBQ_tej_v2_manager.Handler()

更新 CacheHandler 現有因子:   0%|          | 0/251 [00:00<?, ?it/s]

TWN/APIPRCD
close_d
C:\Users\User/Documents\MBQ_tej_v2_DB\TWN/APIPRCD\close_d.parquet
Custom
Common_Stock
C:\Users\User/Documents\MBQ_tej_v2_DB\Custom\Common_Stock.parquet
分布式讀取:C:\Users\User/Documents\MBQ_tej_v2_DB\Custom\Common_Stock.parquet
Start to update雲端 close_d from 2025-04-22 00:00:00 to 2025-04-22 00:00:00
tej_fastget:TWN/APIPRCD:close_d
分布式更新:C:\Users\User\我的雲端硬碟 (owen.lin@mutual-boost.com)\MBQ_Tej_v2\MBQ_tej_v2\TWN/APIPRCD\close_d


Processing files: 100%|██████████| 1/1 [00:00<00:00, 19.88it/s]

Start to update本地 close_d from 2025-04-22 00:00:00 to 2025-04-22 00:00:00
分布式讀取:C:\Users\User\我的雲端硬碟 (owen.lin@mutual-boost.com)\MBQ_Tej_v2\MBQ_tej_v2\TWN/APIPRCD\close_d


分布式讀取:C:\Users\User/Documents\MBQ_tej_v2_DB\TWN/APIPRCD\close_d.parquet


...

分布式讀取:C:\Users\User/Documents\MBQ_tej_v2_DB\TWN/APIPRCD\pbr.parquet


更新 CacheHandler 現有因子:  69%|██████▉   | 173/251 [14:31<04:12,  3.24s/it] 

[錯誤] 處理因子 EBIT_to_EV 失敗：name 'Cash_and_Cash_Equivalent' is not defined


更新 CacheHandler 現有因子: 100%|██████████| 251/251 [27:26<00:00,  6.56s/it]

所有因子更新完成並寫入 CacheHandler


In [ ]:
df_result = pd.read_excel(save_path,index_col=0)
import fitness
df_result['use_data'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,False)[1])
df_result['len'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,True))
原始大小 = len(df_result)

use_data_df = pd.DataFrame([(k, v) for k, values in df_result.set_index('cal')['use_data'].items() for v in values], columns=['key', 'value'])
use_factor = use_data_df.groupby('value')['key'].apply(list)
use_factor_count = use_factor.str.len().sort_values(ascending=False)
df_result_list = []
top_n = 100
for factor_name in tqdm.tqdm(use_factor_count.index[:]):
    df_result_ = df_result.loc[df_result['cal'].str.contains(factor_name)].sort_values('fitness',ascending=False)
    Top = df_result_.index[:top_n]
    Down = df_result_.index[top_n:]
    if len(Down):
        df_result = df_result.drop(index = Down)
    df_result_list.append(df_result.loc[Top])
best_df_result = pd.concat(df_result_list).drop_duplicates('cal').sort_values('fitness',ascending=False).iloc[:1000]
print(f'因子数{len(best_df_result)}/{原始大小}')
print(f'{best_df_result.fitness.mean()}')

100%|██████████| 14/14 [00:00<00:00, 1497.55it/s]

因子数15/15
330.6774053169944


In [5]:
target = eval(args['TARGET_Expr'],CacheHandler)
exp_ret = target = target[np.isfinite(target)]

In [6]:
expr_list = df_result.sort_values('fitness',ascending=False)['cal'].to_list()
index = exp_ret.stack().index
def get_factor_stack(expr,index,args):
    CacheHandler = Tool.Handler(path = args['Handler_cache_path'],data_type = args['data_type'])
    return eval(expr,CacheHandler).stack(dropna = False).reindex(index = index)
factor_df = pd.DataFrame(np.column_stack(Parallel(n_jobs=-1)(delayed(get_factor_stack)(expr,index,args) for expr in tqdm.tqdm(expr_list))),index=index,columns=expr_list)

100%|██████████| 15/15 [00:00<00:00, 1665.77it/s]


In [7]:
def calc_IC_series(factor: pd.DataFrame, exp_ret: pd.DataFrame) -> pd.Series:
    IC_series = factor.corrwith(exp_ret, axis=1, method='spearman')
    return IC_series.sort_index()

def run_IC_test(IC_series: pd.Series, insample_start_date: str, insample_end_date: str) -> pd.DataFrame:
    insample_start = pd.to_datetime(insample_start_date)
    insample_end = pd.to_datetime(insample_end_date)
    outsample_start = pd.to_datetime("2020-01-01")

    IC_mean = pd.Series({
        f"樣本內({insample_start.date()}~{insample_end.date()})": IC_series.loc[insample_start:insample_end].mean(),
        f"樣本外({outsample_start.date()}~{IC_series.index[-1].date()})": IC_series.loc[outsample_start:].mean(),
    })

    ICIR = pd.Series({
        f"樣本內({insample_start.date()}~{insample_end.date()})": IC_mean[0] / IC_series.loc[insample_start:insample_end].std(),
        f"樣本外({outsample_start.date()}~{IC_series.index[-1].date()})": IC_mean[1] / IC_series.loc[outsample_start:].std(),
    })

    return pd.concat({"IC": IC_mean, "ICIR": ICIR}, axis=1)

In [8]:
# 因子檢定模式 = True
# if 因子檢定模式:
#     IC_Se = calc_IC_series(factor_df
#                            ['非负化处理(abs(truediv(signedpower(非负化处理(cs_scale(abs(truediv(signedpower(非负化处理(cs_scale(daily_EMA20, 10)), 30), Volume)), 30)), 3), FAC)))'].unstack(), exp_ret)
#     display(run_IC_test(IC_Se, insample_start_date='2015-01-01', insample_end_date='2019-12-31'))
#     Tool.backtest_factor(factor_df
#                            ['非负化处理(abs(truediv(signedpower(非负化处理(cs_scale(abs(truediv(signedpower(非负化处理(cs_scale(daily_EMA20, 10)), 30), Volume)), 30)), 3), FAC)))'].unstack(), exp_ret, rank_range_n=10, start_date='2019-12-31')
#     IC_Se.ewm(span=60).mean().dropna().iloc[60:].iplot(hline=[IC_Se.mean()])

In [9]:
factor_df.to_pickle('factor_df.pkl')

# 因子整体等权监控

In [10]:
import os
from datetime import time
from datetime import datetime
import numpy as np
import pandas as pd
import json
import pathlib
import tqdm
import Tool
import fitness
def 剔除高度相关(df_result,ts_result,Corr_threshold:float = 0.95):
    要删除的清单 = list()
    #根據因子本身序列評價單因子績效並排序
    #cv_df = ts_result.mean() / ts_result.std()
    #ts_result = ts_result[cv_df.sort_values(ascending = False).index]

    Corr_df = ts_result.corr().abs()
    for factor_expr in ts_result.columns:
        if factor_expr not in 要删除的清单:
            Corr_ = Corr_df.loc[factor_expr:,factor_expr].iloc[1:]
            要删除的清单+=Corr_[Corr_>Corr_threshold].index.to_list()
    要删除的清单 = list(set(要删除的清单))
    if 要删除的清单:
        print(f'发现:{len(要删除的清单)}个表达式与其他其他表达式相关超过{Corr_threshold}，执行删除！！！')
    else:
        print(f'没有相关超过{Corr_threshold}的表达式！！！')
    df_result = df_result.loc[~df_result['cal'].isin(要删除的清单)]
    ts_result = ts_result[df_result['cal'].to_list()]
    return df_result,ts_result

start_time = datetime.now()
print(f"Init variables:{start_time}.")
args = json.load(open("./config.json", encoding="utf-8"))
if args['OUTPUT_DATA_PATH'][:2] == './':
    args['OUTPUT_DATA_PATH'] = str(pathlib.Path().absolute())+args['OUTPUT_DATA_PATH'][1:]
# Data preprocessing
Handler_cache_path = args['Handler_cache_path']
print(f"Load Data : {Handler_cache_path}")
CacheHandler = Tool.Handler(path = Handler_cache_path,data_type = args['data_type'])

Con = eval(args['Con'],CacheHandler)
target = eval(args['TARGET_Expr'],CacheHandler)
target = target[np.isfinite(target) & Con]
print(f"Target : ({args['TARGET_Expr']})[{args['Con']}]")

使用太多次需剔除的data_name = []
if os.path.exists(args['OUTPUT_DATA_PATH']):
    ts_result_path = os.path.join(os.path.dirname(args['OUTPUT_DATA_PATH']),f"{args['OUTPUT_DATA_PATH'].split('/')[-1].replace('.xlsx','')}.parquet")
    ts_result = pd.read_parquet(ts_result_path)
    df_result = pd.read_excel(args['OUTPUT_DATA_PATH'],index_col=0)
    df_result,ts_result = 剔除高度相关(df_result,ts_result,args['Corr_threshold'])
    # 开始确认源资料使用频繁程度
    df_result['use_data'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,False)[1])
    df_result = df_result.loc[df_result['fitness']>args['FIT_THRES']]
    print(f"{args['OUTPUT_DATA_PATH']} 已存在，载入历史{len(df_result)}个历史因子")
    use_data_df = pd.DataFrame([(k, v) for k, values in df_result.set_index('cal')['use_data'].items() for v in values], columns=['key', 'value'])
    use_factor = use_data_df.groupby('value')['key'].apply(list)
    use_factor_count = use_factor.str.len().sort_values(ascending=False)
    使用太多次需剔除的data_name = use_factor_count.loc[use_factor_count>=args['use_data_count_threshold']].index.to_list()
    if 使用太多次需剔除的data_name:
        print(f'剔除:{使用太多次需剔除的data_name}')
    print(f'Mega Factor Score Up to:{ts_result.mean(axis = 1).mean() / ts_result.mean(axis = 1).std() * 252**0.5}')
else:
    df_result = pd.DataFrame(columns=['cal', 'fitness', 'use_data'])
    ts_result = pd.DataFrame()
count = 0
if args['AMT']=="None" or args['AMT']=="inf": args['AMT'] = np.inf


def factor_to_weight(alpha_array,only_long:bool = False):
    demeaned = alpha_array - np.nanmean(alpha_array,axis = 1)[:, None]
    weights = demeaned / np.nansum(np.abs(demeaned),axis = 1)[:, None]
    weights[np.isnan(weights)] = 0#检查数据无值时视为权重0
    if only_long:
        weights[weights<0] = 0
        weights*=2
    return weights

Full_time_ts_result = pd.concat({expr:factor_to_weight(eval(expr,CacheHandler)[target.notna()],True).mul(CacheHandler['Adj_close'].shift(-2) / CacheHandler['Adj_close'].shift(-1) -1).sum(axis = 1) for expr in tqdm.tqdm(df_result['cal'][:])},axis=1)

Init variables:2025-04-21 10:18:07.608703.
Load Data : C:\Users\User\Documents\MBQ_TW_DB
Target : (Adj_close.shift(-21) / Adj_close.shift(-1) - 1)[(Value_Dollars.rolling(5).mean() > 0)]
没有相关超过0.8的表达式！！！
c:\Users\User\Stock-Futures-Portfolio/Results/Results.xlsx 已存在，载入历史15个历史因子
Mega Factor Score Up to:11.239586854105067


100%|██████████| 15/15 [00:16<00:00,  1.07s/it]


In [11]:
# top15_expr_list = df_result['cal'].head(15).tolist()
# print("前15個因子如下：")
# print(top15_expr_list)

# # 對前 15 個因子進行回測計算
# Full_time_ts_result = pd.concat({
#     expr: factor_to_weight(eval(expr, CacheHandler)[target.notna()], only_long=True)
#     .mul(CacheHandler['Adj_close'].shift(-2) / CacheHandler['Adj_close'].shift(-1) - 1)
#     .sum(axis=1)
#     for expr in tqdm.tqdm(top15_expr_list)
# }, axis=1)

In [12]:
bt_ret = Full_time_ts_result.mean(axis=1)
Tool.show_stats(pd.concat({
    '樣本內':bt_ret.loc[args['DATE_FROM']:args['DATE_TO']],
    '樣本外':bt_ret.loc[args['DATE_TO']:],
    '全樣本':bt_ret.loc[:],
},axis = 1))
bt_ret.cumsum().iplot(vline = [args['DATE_FROM'],args['DATE_TO']])

,CAGR(%),Sharpe,Calmar,MDD(%),單利MDD(%),样本胜率(%),周胜率(%),月胜率(%),年胜率(%),盈亏比(avg_win/avg_loss),总赚赔比(profit_factor),预期报酬(bps),样本数
樣本內,110.12,8.23,8.48,-12.98,-2.61,70.27,81.45,99.07,100.0,1.67,3.98,43.99,2223
樣本外,34.45,5.28,2.45,-14.09,-6.21,63.15,72.04,89.47,100.0,1.43,2.54,17.51,1525
全樣本,182.51,7.05,12.95,-14.09,-2.61,67.37,77.57,95.11,100.0,1.59,3.35,61.58,3748


In [13]:

combined_list = [
    'PBR_TWSE','Total_Operating_Income','Earnings_Before_Interest_and_Tax','Total_Liabilities','Cash_and_Cash_Equivalent','Total_Operating_Income','Basic_Earnings_Per_Share','Return_Rate_on_Equity_A_percent',
    'Total_Assets','Sales_Monthly','Cash_Flow_from_Operating_Activities','Cash_Dividend_Yield_TEJ','Return_on_Total_Assets_A_percent','Net_Operating_Income_Loss'
]


In [14]:
# first = pd.read_parquet('./Results/v4.parquet')
# second = pd.read_parquet('./Results/Results.parquet')
# final = pd.concat([first,second],axis = 1)
# final.to_parquet('./Results/Results.parquet')

In [15]:
manage = MBQ_tej_v2_manager.Handler()

In [16]:
# def manage_factors(CacheHandler):
#     for factor_name in combined_list:
#         if factor_name in CacheHandler.cash_list():
#             continue

#         # 讀入資料並 pivot
#         factor_data = manage(factor_name)
#         if 'mdate' in factor_data.index.names and len(factor_data.columns) > 1:
#             all_factor = factor_data
#         else:
#             Q_columns = [col for col in factor_data.columns if col.endswith('_Q')]
#             required_columns = ['mdate', 'coid'] + Q_columns
#             factor_data_Q = factor_data[required_columns]
#             all_factor = factor_data_Q.pivot(index='mdate', columns='coid', values=Q_columns)
#             if isinstance(all_factor.columns, pd.MultiIndex):
#                 all_factor.columns = all_factor.columns.droplevel(0)

#         all_factor_filtered = all_factor
#         CacheHandler[factor_name] = all_factor_filtered

#     return CacheHandler
# manage_factors(CacheHandler)

In [17]:
def manage_factors(CacheHandler):
    factor_dict = {}

    def add_factor(factor_name, formula):
        factor_dict[factor_name] = formula
        CacheHandler[factor_name] = formula

    add_factor('52_high', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].rolling(252).max())
    add_factor('EP', CacheHandler['Basic_Earnings_Per_Share'] / CacheHandler['Adj_Close'])
    EV = CacheHandler['Value_Dollars'] + CacheHandler['Total_Liabilities'] - CacheHandler['Cash_and_Cash_Equivalent']
    add_factor('EBIT_to_EV', CacheHandler['Earnings_Before_Interest_and_Tax'] / EV)
    bp_quantile = CacheHandler['PBR_TWSE'].rank(pct=True).apply(lambda x: 1 / x)
    roe_quantile = CacheHandler['Return_Rate_on_Equity_A_percent'].rank(pct=True).apply(lambda x: 1 / x)
    add_factor('BPROE_Quantile', bp_quantile + roe_quantile)
    add_factor('Log PS', CacheHandler['Total_Operating_Income'] / CacheHandler['Value_Dollars'].apply(np.log))
    add_factor('One-month change', (CacheHandler['Adj_Close'] - CacheHandler['Adj_Close'].shift(20)) / CacheHandler['Adj_Close'].shift(20))
    add_factor('MoM5', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(5) - 1)
    add_factor('MoM20', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(20) - 1)
    add_factor('MoM60', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(60) - 1)
    add_factor('MoM252', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(252) - 1)
    add_factor('Turn20', CacheHandler['Turnover'].rolling(20).mean())
    add_factor('PctTurn', CacheHandler['Turnover'] / CacheHandler['Turnover'].rolling(40).mean().shift(20) - 1)
    add_factor('PctTurn20', (CacheHandler['Turnover'] / CacheHandler['Turnover'].rolling(40).mean().shift(20) - 1).rolling(20).mean())
    add_factor('振幅', (CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('振幅換手率因子', ((CacheHandler['Turnover'] / CacheHandler['Turnover'].rolling(40).mean().shift(20) - 1) * ((CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))).rolling(20).mean())
    add_factor('STR', CacheHandler['Turnover'].rolling(20).std())
    add_factor('Vol20', (CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift() - 1).rolling(20).std())
    add_factor('SCR', CacheHandler['Turnover'].rolling(20).std() / CacheHandler['Turnover'].rolling(40, min_periods=1).std().shift(20) - 1)
    add_factor('DRET', (CacheHandler['Adj_Close'] - CacheHandler['Adj_Open']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('FAC', (CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('PLUS', (2 * CacheHandler['Adj_Close'] - CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('daily_turnover_change', CacheHandler['Turnover'] / CacheHandler['Turnover'].shift(1) - 1)
    add_factor('daily_turnover_change_ma20', (CacheHandler['Turnover'] / CacheHandler['Turnover'].shift(1) - 1).rolling(20).mean())
    add_factor('gtr', (CacheHandler['Turnover'] / CacheHandler['Turnover'].shift(1) - 1).rolling(20).std())
    intraday_ret = (CacheHandler['Adj_Close'] - CacheHandler['Adj_Open']) / CacheHandler['Adj_Open']
    add_factor('intraday_return_ewm_vol20', intraday_ret.pct_change().ewm(span=20).mean())

    return factor_dict

factor_dict = manage_factors(CacheHandler)


In [18]:
factor_cols = [col for col in factor_df.columns if col not in ['mdate', 'coid']]
factor_df.reset_index(inplace=True)
factor_dict_gene = {
    factor: factor_df.pivot(index='mdate', columns='coid', values=factor)
    for factor in factor_cols
}

In [19]:
final_factor_dict = {**factor_dict, **factor_dict_gene}

In [20]:
import bottleneck as bn

In [21]:
class FactorCombination:
    def __init__(self, stock_return=None):
        """
        Initialize the FactorCombination class.

        Parameters:
        - stock_return: DataFrame, optional
            Stock return data, typically in decimal format.
        """
        self.stock_return = stock_return

    @staticmethod
    def get_stock_return_close_to_close(daily_return_data):
        """
        Calculate close-to-close stock return from daily percentage return data.

        Parameters:
        - daily_return_data: DataFrame
            Daily percentage return data.

        Returns:
        - DataFrame: Stock returns as decimal values.
        """
        return daily_return_data.div(100)

    @staticmethod
    def get_demean_weighting(data):
        """
        Calculate demeaned and normalized weighting.

        Parameters:
        - data: DataFrame
            Input factor data.

        Returns:
        - DataFrame: Demeaned and normalized weighting.
        """
        df = data.dropna(axis=0, how='all').dropna(axis=1, how='all').copy()
        demean = df.sub(df.mean(axis=1, skipna=True), axis=0)
        weighting = demean.div(demean.abs().sum(axis=1, min_count=1, skipna=True), axis=0)
        weighting = weighting.replace([np.inf, -np.inf], np.nan)
        return weighting

    def combine_factor(self, factor_dict, method='equal_weight', period=250, std_power=1, half_life=126):
        """
        Combine multiple factors using the specified method.

        Parameters:
        - factor_dict: dict
            Dictionary of factors to combine.
        - method: str
            Combination method ('equal_weight', 'double_momentum', 'IC_IR_model', or 'PCA').
        - period: int
            Rolling period for momentum calculation (used in 'double_momentum').
        - std_power: int
            Exponent for standard deviation in momentum weighting (used in 'double_momentum').

        Returns:
        - DataFrame: Combined factor.
        """
        if not factor_dict:
            raise ValueError("factor_dict is empty. Ensure that factors are properly calculated before combining.")

        if method == 'equal_weight':
            multi_factor_weighting = self.get_demean_weighting(list(factor_dict.values())[0]).fillna(0)
            for factor in list(factor_dict.values())[1:]:
                demean_factor = self.get_demean_weighting(factor).fillna(0)
                multi_factor_weighting = multi_factor_weighting.add(demean_factor, fill_value=0)
            factor_all = multi_factor_weighting

        elif method == 'double_momentum':
            if self.stock_return is None:
                raise ValueError("stock_return is required for 'double_momentum' method.")
            factor_all = self.double_momentum_model(factor_dict=factor_dict, period=period, std_power=std_power)
        elif method == 'IC_IR_model':
            if self.stock_return is None:
                raise ValueError("stock_return is required for 'IC_IR' method.")
            factor_all = self.IC_IR_model(factor_dict=factor_dict, stock_return=self.stock_return, period=period)
        elif method == 'PCA':
            factor_all = self.pca_model(factor_dict=factor_dict)
        elif method == 'half_ic_ir_model':
            if self.stock_return is None:
                raise ValueError("stock_return is required for 'half_ic_ir_model' method.")
            factor_all = self.half_ic_ir_model(factor_dict=factor_dict, period=period, half_life=half_life)   
        elif method == 'XGBoost':
            factor_all = self.xgboost_model(factor_dict=factor_dict)
        else:
            raise ValueError(f"Unknown method '{method}'. Please choose 'equal_weight', 'double_momentum', 'IC_IR_model', or 'half_ic_ir_model'.")

        return factor_all

    def double_momentum_model(self, factor_dict, period=252, std_power=1):
        """
        Double Momentum Model for combining multiple factors without auto direction adjustment.

        Parameters:
        - factor_dict: dict
            Dictionary of factors to combine.
        - period: int
            Rolling period for momentum calculation.
        - std_power: int
            Exponent for standard deviation in momentum weighting.

        Returns:
        - DataFrame: Combined factor with momentum-based weighting.
        """
        print("Step 1: Calculating demean weights...")
        demean_weight_dict = {k: self.get_demean_weighting(v).fillna(0) for k, v in factor_dict.items()}
        for k, v in demean_weight_dict.items():
            print(f"Demeaned weights for factor {k}:\n{v.head()}")

        print("\nStep 2: Calculating long-short returns...")
        ret_ls_dict = {
            k: (w.shift(2) * self.stock_return).sum(axis=1, min_count=1)
            for k, w in demean_weight_dict.items()
        }
        for k, v in ret_ls_dict.items():
            print(f"Long-short returns for factor {k}:\n{v.head()}")

        print("\nStep 3: Preparing df_ret_ls...")
        df_ret_ls = pd.DataFrame(ret_ls_dict).dropna(how='all')
        print("df_ret_ls:\n", df_ret_ls.head())

        print("\nStep 4: Calculating rolling mean and score...")
        rolling_mean_ret = pd.DataFrame(
            bn.move_mean(df_ret_ls.values, window=period, axis=0, min_count=1),
            index=df_ret_ls.index, columns=df_ret_ls.columns
        )
        rolling_std_ret = pd.DataFrame(
            bn.move_std(df_ret_ls.values, window=period, axis=0, min_count=1),
            index=df_ret_ls.index, columns=df_ret_ls.columns
        )
        rolling_score = rolling_mean_ret / (rolling_std_ret ** std_power + 1e-10)
        print("Rolling mean returns:\n", rolling_mean_ret.head())
        print("Rolling score:\n", rolling_score.head())

        print("\nStep 5: Calculating normalized weights...")
        score_weight = (rolling_score.abs().T / rolling_score.abs().sum(axis=1, min_count=1)).T.fillna(0)
        direction = np.sign(rolling_mean_ret).fillna(0)
        factor_weight = score_weight * direction
        factor_weight = factor_weight.clip(lower=0) * 2  # long-only + boost
        print("Factor weight (adjusted):\n", factor_weight.head())

        print("\nStep 6: Combining factors...")
        combined_factor = pd.DataFrame(0, index=self.stock_return.index, columns=self.stock_return.columns)
        for k in demean_weight_dict:
            w = factor_weight[k].reindex(demean_weight_dict[k].index)
            df = demean_weight_dict[k]
            df = df.loc[w.index.intersection(df.index)]
            combined = df.mul(w, axis=0)
            combined_factor = combined_factor.add(combined, fill_value=0)

        print("Final merged factor:\n", combined_factor.head())
        return combined_factor

In [22]:
fc = FactorCombination(stock_return=CacheHandler['Adj_close']/ CacheHandler['Adj_close'].shift() -1)
ic_ir = fc.combine_factor(factor_dict=final_factor_dict, method='double_momentum', period=20)

Step 1: Calculating demean weights...
Demeaned weights for factor 52_high:
coid            1101      1102      1103      1104      1107      1108  \
mdate                                                                    
2011-01-03  0.001036  0.001135  0.001179  0.000455  0.000167  0.000567   
2011-01-04  0.000980  0.001206  0.001083  0.000570  0.000508  0.000550   
2011-01-05  0.000907  0.001200  0.001062  0.000587  0.000971  0.000540   
2011-01-06  0.000876  0.001148  0.001067  0.000572  0.001247  0.000496   
2011-01-07  0.000899  0.001188  0.000891  0.000459  0.001342  0.000433   

coid            1109      1110      1201      1203  ...      9944      9945  \
mdate                                               ...                       
2011-01-03  0.001152  0.001212  0.000628  0.000969  ...  0.001071  0.000488   
2011-01-04  0.001176  0.001137  0.000490  0.000886  ...  0.001194  0.000547   
2011-01-05  0.001124  0.000973  0.000444  0.000853  ...  0.001243  0.000498   
2011-01-06 

In [23]:
futures = pd.read_csv('20250421101138.csv', encoding='utf-16', sep='\t')
futures['年月日'] = pd.to_datetime(futures['年月日'].astype(str))
futures = futures.sort_values(['標的證券', '證券代碼', '年月日'])

futures['contract_month'] = futures['證券代碼'].str.extract(r'(\d{6})')[0]
futures['contract_month'] = pd.to_datetime(futures['contract_month'], format='%Y%m', errors='coerce')
futures = futures.dropna(subset=['contract_month'])
futures['coid'] = futures['標的證券'].astype(str).str.extract(r'([0-9]{4})')

# === 計算轉倉日（每月第三個週三的前一天）===
def get_third_wednesday_dates(start_date, end_date):
    all_months = pd.date_range(start=start_date, end=end_date, freq='MS')
    third_wednesdays = []
    for first_day in all_months:
        wednesdays = pd.date_range(first_day, first_day + pd.offsets.MonthEnd(0), freq='W-WED')
        if len(wednesdays) >= 3:
            third_wednesdays.append(wednesdays[2])
    return pd.to_datetime(third_wednesdays)

trade_dates = futures['年月日'].drop_duplicates().sort_values()
third_wed = get_third_wednesday_dates(trade_dates.min(), trade_dates.max())
roll_dates = pd.to_datetime([w - pd.Timedelta(days=1) for w in third_wed])
roll_dates_set = set(roll_dates)
roll_dict = {(d.year, d.month): d for d in roll_dates}

def assign_target_contract_month(date):
    year, month = date.year, date.month
    if date in roll_dates_set:
        return pd.Timestamp(f"{year+1}-01-01") if month == 12 else pd.Timestamp(f"{year}-{month+1:02d}-01")
    rd = roll_dict.get((year, month))
    if rd and date > rd:
        return pd.Timestamp(f"{year+1}-01-01") if month == 12 else pd.Timestamp(f"{year}-{month+1:02d}-01")
    return pd.Timestamp(f"{year}-{month:02d}-01")

futures['target_contract_month'] = futures['年月日'].apply(assign_target_contract_month)

# === 選出合約 ===
valid = futures[futures['contract_month'] == futures['target_contract_month']]
idx = valid.groupby(['年月日', 'coid'])['成交張數(量)'].idxmax()
top_contracts = valid.loc[idx].sort_values(['年月日', 'coid']).reset_index(drop=True)

adj_factor = CacheHandler['Adjust_Factor']  
adj_factor.index.name = 'mdate'  
adj_factor = adj_factor.reset_index().melt(id_vars='mdate', var_name='coid', value_name='adjust_factor')
adj_factor = adj_factor.rename(columns={'mdate': 'date'})
adj_factor['date'] = pd.to_datetime(adj_factor['date'])

top_contracts['coid'] = top_contracts['標的證券'].str.extract(r'(\d{4})')
top_contracts['date'] = top_contracts['年月日']
top_contracts = top_contracts.merge(adj_factor, on=['date', 'coid'], how='left')

top_contracts['調整係數'] = top_contracts['adjust_factor'].fillna(1.0)
top_contracts['復權收盤價'] = top_contracts['收盤價'] * top_contracts['調整係數']
top_contracts['復權開盤價'] = top_contracts['開盤價'] * top_contracts['調整係數']
df = top_contracts.copy()
df = df[['date', 'coid', '報酬率', '成交張數(量)', '復權收盤價','復權開盤價']].dropna(subset=['復權收盤價'])
price_matrix = df.pivot(index='date', columns='coid', values='復權收盤價').reindex(columns=ic_ir.columns)
open_matrix = df.pivot(index='date', columns='coid', values='復權開盤價').reindex(columns=ic_ir.columns)
exp_ret_fu = price_matrix.shift(-2) / price_matrix.shift(-1) - 1

availability_matrix = (
    df.assign(value=1)
    .pivot_table(index='date', columns='coid', values='value', fill_value=0)
    .astype(bool)
    .reindex(columns=ic_ir.columns, fill_value=0)
)

volume_matrix = df.pivot(index='date', columns='coid', values='成交張數(量)').fillna(0).reindex(columns=ic_ir.columns, fill_value=0)
con_mask = volume_matrix.rolling(20, min_periods=5).mean() > 200
mask = availability_matrix & con_mask

### check

In [ ]:
# def check_roll_contract(coid, signal_date):
#     signal_date = pd.to_datetime(signal_date)
#     trade_dates = top_contracts['年月日'].sort_values().unique()

#     # 定義進出場日
#     try:
#         signal_idx = list(trade_dates).index(signal_date)
#         entry_date = trade_dates[signal_idx + 1]
#         exit_date = trade_dates[signal_idx + 2]
#     except IndexError:
#         print("交易日不足，無法計算 t+1, t+2")
#         return

#     for d in [signal_date, entry_date, exit_date]:
#         row = top_contracts[
#             (top_contracts['年月日'] == d) &
#             (top_contracts['標的證券'].astype(str).str.contains(coid))
#         ]
#         if row.empty:
#             print(f"{d.date()} 無對應合約")
#         else:
#             code = row.iloc[0]['證券代碼']
#             cm = row.iloc[0]['contract_month'].strftime("%Y-%m")
#             print(f"{d.date()} 使用合約：{code}，contract_month: {cm}")


In [ ]:
# check_roll_contract("世紀鋼", "2025-04-17")  # 假設這是轉倉週的週二

交易日不足，無法計算 t+1, t+2


In [26]:
# example_stock = '9958'
# check_df = top_contracts[top_contracts['標的證券'].astype(str).str.contains(example_stock)]
# check_df = check_df[['年月日', '證券代碼', 'contract_month', 'target_contract_month']].sort_values('年月日')
# check_df[:60]

In [27]:
def max_drawdown(prices):
    # 計算累計的最大值
    cumulative_max = prices.cummax()
    # 計算回撤 (Drawdown)
    drawdown = (prices - cumulative_max) / cumulative_max
    # 計算最大回撤 (MDD)
    mdd = drawdown.min()
    return mdd
def show_bt(bt_ret):
    display(pd.concat({"CAGR(%)":bt_ret.mean() * 252,
            'Sharpe' : bt_ret.mean() / bt_ret.std()*252**0.5,
            'Calmar':bt_ret.calmar(),
            'MDD(%)' : bt_ret.max_drawdown()*100,
            '單利MDD(%)' : max_drawdown(bt_ret.cumsum().add(1))*100,
            '樣本勝率(%)' : bt_ret.apply(lambda X:((X.dropna()>0).sum() / X.dropna().shape[0])*100),
            '周勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('W').prod().sub(1)>0).sum() / X.dropna().add(1).resample('W').prod().sub(1).dropna().shape[0])*100),
            '月勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('ME').prod().sub(1)>0).sum() / X.dropna().add(1).resample('ME').prod().sub(1).shape[0])*100),
            '年勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('YE').prod().sub(1)>0).sum() / X.dropna().add(1).resample('YE').prod().sub(1).shape[0])*100),
            'avg_win/avg_loss' : bt_ret.apply(lambda X:(X[X > 0].mean() / abs(X[X < 0].mean()))),
            'profit_factor' : bt_ret.profit_factor(),
            },axis = 1).round(2))
def backtest_factor(factor:pd.DataFrame,exp_ret:pd.DataFrame,rank_range_n:int = 10,start_date:str = '2015-01-01'):
    factor_rank = factor.rank(axis = 1,pct = True,method = 'first')

    IC_Se = factor.corrwith(exp_ret,axis=1,method='spearman').sort_index().loc[start_date:]
    print(f'IC_mean:{round(IC_Se.mean(),4)}')
    print(f'IC_IR:{round(IC_Se.mean()/IC_Se.std(),4)}')

    bt = pd.concat({f'{int(((_/rank_range_n)*100))}% ~ {int((_+1)/rank_range_n*100)}%':exp_ret[(factor_rank>_/rank_range_n) & (factor_rank<=(_+1)/rank_range_n)].mean(axis = 1) - exp_ret.mean(axis=1) for _ in tqdm.tqdm(range(rank_range_n))}, axis = 1).dropna(how = 'all')
    bt = bt.loc[start_date:]
    if (bt.iloc[:,-1] - bt.iloc[:,0]).add(1).prod() > 1:
        bt['LS_ret'] = bt.iloc[:,-1] - bt.iloc[:,0]
    else:
        bt['LS_ret'] = bt.iloc[:,0] - bt.iloc[:,-1]
    show_bt(bt)

    (bt.drop(columns='LS_ret').loc[start_date:].cagr()*100).iplot(kind = 'bar')
backtest_factor(ic_ir,CacheHandler['Adj_Close'].shift(-2) / CacheHandler['Adj_Close'].shift(-1) -1 ,rank_range_n = 10,start_date = '2020-01-01')

IC_mean:-0.0112
IC_IR:-0.0852


100%|██████████| 10/10 [00:01<00:00,  5.53it/s]


,CAGR(%),Sharpe,Calmar,MDD(%),單利MDD(%),樣本勝率(%),周勝率(%),月勝率(%),年勝率(%),avg_win/avg_loss,profit_factor
0% ~ 10%,-0.02,-0.27,-0.08,-19.69,-19.59,47.70,48.74,45.31,16.67,1.04,0.95
10% ~ 20%,-0.05,-0.95,-0.13,-25.79,-28.69,46.37,42.60,42.19,0.00,0.98,0.85
20% ~ 30%,-0.04,-0.89,-0.12,-19.33,-21.15,46.84,42.24,31.25,16.67,0.97,0.86
30% ~ 40%,-0.02,-0.78,-0.12,-14.52,-15.37,45.67,44.04,39.06,0.00,1.04,0.88
40% ~ 50%,0.00,0.14,0.04,-5.96,-5.68,49.30,47.29,42.19,66.67,1.05,1.02
50% ~ 60%,0.00,0.08,0.02,-10.53,-10.74,48.81,48.01,50.00,50.00,1.06,1.02
60% ~ 70%,-0.03,-0.42,-0.11,-18.66,-18.92,48.49,42.96,50.00,16.67,0.97,0.91
70% ~ 80%,0.07,0.42,0.12,-26.06,-27.29,50.83,50.18,53.12,66.67,1.10,1.14
80% ~ 90%,-0.01,-0.06,-0.03,-25.56,-25.70,53.31,46.21,54.69,50.00,0.86,0.99
90% ~ 100%,0.10,1.04,0.49,-13.07,-11.05,52.85,53.43,54.69,83.33,1.07,1.20


In [28]:
Top_n =15 #選股數量

In [30]:
from datetime import datetime
import tejapi
benchmark = tejapi.get('TWN/APIPRCD', coid='IX0001',opts={'columns': ['mdate','close_d']}, mdate={'gte': '2010-01-01', 'lte':'2025-04-18'}, paginate=True)
benchmark["mdate"] = pd.to_datetime(benchmark["mdate"])
benchmark["mdate"] = benchmark["mdate"].dt.strftime("%Y-%m-%d")
benchmark_return = benchmark['close_d']
benchmark_return.index = CacheHandler['Adj_Close'].index
bench_exp_ret = benchmark_return.shift(-1)

In [85]:
def calculate_return_series(ic_ir_factor, Top_n, exp_ret, close_px, plot=False, stop_loss_threshold=-0):
    Top_n = int(Top_n)
    stop_loss_records = []
    month_end_dates = ic_ir_factor.groupby(ic_ir_factor.index.to_period("W")).apply(lambda x: x.index.max()).values
    latest_signal_date = month_end_dates[-1]
    print(f"🕒 最新一期更改進場持股的日期是：{latest_signal_date}")
    def fill_signal(signal, weight):
        filled_signal = pd.DataFrame(index=weight.index, columns=weight.columns)
        first_date = month_end_dates[0]
        filled_signal.loc[:first_date] = signal.loc[first_date].values
        for i in range(len(month_end_dates) - 1):
            current_date, next_date = month_end_dates[i], month_end_dates[i + 1]
            date_range = weight.loc[current_date:next_date].index
            filled_signal.loc[date_range] = signal.loc[current_date].values
        return filled_signal.dropna(how='all') > 0

    # Step 1: 產生進場信號
    signal_bool = {
        "ic_ir": (ic_ir_factor.rank(axis=1, ascending=False, method='first') <= Top_n).astype(int),
    }

    filled_signals = {
        k: fill_signal(v, ic_ir_factor if k == "ic_ir" else None)
        for k, v in signal_bool.items()
    }

    W_df = {k: signal.astype(int) / Top_n for k, signal in filled_signals.items()}

    def calculate_return(weighting):
        delta_weight = weighting.diff()
        buy_fee = delta_weight[delta_weight > 0] * (0.001425 * 0.3 + 0.003)
        sell_fee = delta_weight[delta_weight < 0].abs() * (0.001425 * 0.3)

        effective_weight = weighting.copy()

        for i in range(len(month_end_dates) - 1):
            signal_date = month_end_dates[i]
            if signal_date not in weighting.index:
                continue

            idx = weighting.index.get_loc(signal_date)
            if idx + 2 >= len(weighting):  # 需 entry、exit
                continue

            entry_date = weighting.index[idx + 1]
            exit_date = weighting.index[idx + 2]
            active_stocks = weighting.columns[weighting.loc[signal_date] > 0]

            dropped = []
            for stock in active_stocks:
                if (
                    entry_date not in exp_ret.index
                    or exit_date not in exp_ret.index
                    or stock not in exp_ret.columns
                ):
                    continue

                # 累積報酬
                ret_path = exp_ret[stock].loc[entry_date:exit_date].fillna(0)
                cum_ret = (1 + ret_path).cumprod() - 1

                # 若觸發停損
                if (cum_ret < stop_loss_threshold).any():
                    if stock in close_px.columns and entry_date in close_px.index and exit_date in close_px.index:
                        prev_close = close_px.at[entry_date, stock]
                        exit_close = close_px.at[exit_date, stock]
                        limit_threshold = prev_close * 0.91

                        if exit_close <= limit_threshold:
                            # 延後一天出場
                            next_idx = weighting.index.get_loc(exit_date) + 1
                            if next_idx < len(weighting.index):
                                delay_exit = weighting.index[next_idx]
                                dropped.append(stock)
                                stop_loss_records.append((delay_exit, stock))
                                effective_weight.at[delay_exit, stock] = 0

                                remaining = effective_weight.loc[delay_exit, active_stocks].drop(index=dropped, errors='ignore')
                                if not remaining.empty:
                                    effective_weight.loc[delay_exit, remaining.index] = 1.0 / len(remaining)
                            continue

                    # 否則照當天收盤出場
                    dropped.append(stock)
                    stop_loss_records.append((exit_date, stock))
                    effective_weight.at[exit_date, stock] = 0

                    remaining = effective_weight.loc[exit_date, active_stocks].drop(index=dropped, errors='ignore')
                    if not remaining.empty:
                        effective_weight.loc[exit_date, remaining.index] = 1.0 / len(remaining)

        daily_ret_matrix = effective_weight * exp_ret
        daily_ret = daily_ret_matrix.sum(axis=1)
        cost = buy_fee.fillna(0).sum(axis=1) + sell_fee.fillna(0).sum(axis=1)
        return daily_ret - cost, daily_ret_matrix, effective_weight

    weightings = {k: W_df[k] for k in W_df}
    return_series = {}
    ret_matrix_dict = {}
    effective_weight_dict = {}

    for k, w in weightings.items():
        r, r_matrix, e_weight = calculate_return(w)
        return_series[k] = r
        ret_matrix_dict[k] = r_matrix
        effective_weight_dict[k] = e_weight

    if plot:
        for k, v in return_series.items():
            v.iplot(title=k)

    bt_ret = pd.concat({
        'ic_ir_return': return_series['ic_ir'],
    }, axis=1).loc['2020':]

    return bt_ret, weightings, stop_loss_records, ret_matrix_dict, effective_weight_dict

bt_ret, weightings, stop_loss_records, ret_matrix_dict, effective_weight_dict = calculate_return_series(
    ic_ir_factor=ic_ir[mask].loc['2020':],
    Top_n=10,
    exp_ret=exp_ret_fu.loc['2020':],
    close_px=price_matrix.loc['2020':],  
    plot=False
)


🕒 最新一期更改進場持股的日期是：2025-04-18T00:00:00.000000000


In [98]:
signal_dates = weightings['ic_ir'].index.to_series().groupby(lambda x: x.to_period("W")).apply(lambda x: x.max()).values

# 最新一期與前一期
latest_date = signal_dates[-3]
previous_date = signal_dates[-2]

In [99]:
# 取出進場信號（權重 > 0）的股票
latest_stocks = weightings['ic_ir'].loc[latest_date]
latest_stocks = latest_stocks[latest_stocks > 0].index

previous_stocks = weightings['ic_ir'].loc[previous_date]
previous_stocks = previous_stocks[previous_stocks > 0].index


In [ ]:
latest_set = set(latest_stocks)
previous_set = set(previous_stocks)

added = latest_set - previous_set       # 新增
removed = previous_set - latest_set     # 被剔除
unchanged = latest_set & previous_set   # 持續持有

print(f"前一期調倉日：{previous_date}")
print(f"最新一期調倉日：{latest_date}")
print("\n新增股票：", sorted(added))
print("剔除股票：", sorted(removed))
print("持續持有：", sorted(unchanged))


📅 前一期調倉日：2025-04-11T00:00:00.000000000
📅 最新一期調倉日：2025-04-02T00:00:00.000000000

✅ 新增股票： ['1319', '2474', '2609', '2615', '3260', '3264', '5534']
❌ 剔除股票： ['2317', '2330', '2382', '2454', '3231', '5274', '6669']
🔁 持續持有： ['2603', '2634', '3481']


In [ ]:
stop_loss_df = pd.DataFrame(stop_loss_records, columns=['exit_date', 'stock'])
stop_loss_df = stop_loss_df.sort_values('exit_date')
stop_loss_df.tail(20)  

,exit_date,stock
1583,2025-04-08,5534
1574,2025-04-08,1319
1575,2025-04-08,2474
1576,2025-04-08,2603
1579,2025-04-08,2634
1582,2025-04-09,3481
1581,2025-04-09,3264
1578,2025-04-09,2615
1577,2025-04-09,2609
1580,2025-04-09,3260


In [73]:
import warnings
warnings.filterwarnings("ignore")
def max_drawdown(prices):
    # 計算累計的最大值
    cumulative_max = prices.cummax()
    # 計算回撤 (Drawdown)
    drawdown = (prices - cumulative_max) / cumulative_max
    # 計算最大回撤 (MDD)
    mdd = drawdown.min()
    return mdd
display(pd.concat({"CAGR(%)":bt_ret.mean() * 252 * 100,
        'Sharpe' : bt_ret.mean() / bt_ret.std()*252**0.5,
        'Calmar':bt_ret.cagr() / abs(bt_ret.max_drawdown()),
        'MDD(%)' : bt_ret.max_drawdown()*100,
        '單利MDD(%)' : max_drawdown(bt_ret.cumsum().add(1))*100,
        '樣本勝率(%)' : bt_ret.apply(lambda X:((X.dropna()>0).sum() / X.dropna().shape[0])*100),
        '周勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('W').prod().sub(1)>0).sum() / X.dropna().add(1).resample('W').prod().sub(1).dropna().shape[0])*100),
        '月勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('ME').prod().sub(1)>0).sum() / X.dropna().add(1).resample('ME').prod().sub(1).shape[0])*100),
        '年勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('YE').prod().sub(1)>0).sum() / X.dropna().add(1).resample('YE').prod().sub(1).shape[0])*100),
        "avg_win(%)": bt_ret.apply(lambda X: X[X > 0].mean() * 100),
        "avg_loss(%)": bt_ret.apply(lambda X: X[X < 0].mean() * 100),
        'avg(%)':bt_ret.apply(lambda X:X.mean() * 100),
        'avg_win/avg_loss' : bt_ret.apply(lambda X:(X[X > 0].mean() / abs(X[X < 0].mean()))),
        'profit_factor' : bt_ret.profit_factor(),
        },axis = 1).round(2))
bt_ret.cumsum().ffill().iplot(title='Equity Curve')
bt_ret.to_drawdown_series().iplot(title = 'Drawdown')
pd.concat({
    f'equal({round(weightings["ic_ir"].loc["2020":].sum(axis=1).mean(), 4)})': weightings['ic_ir'].loc["2020":].sum(axis=1),
}, axis=1).iplot(title='現金使用率')


,CAGR(%),Sharpe,Calmar,MDD(%),單利MDD(%),樣本勝率(%),周勝率(%),月勝率(%),年勝率(%),avg_win(%),avg_loss(%),avg(%),avg_win/avg_loss,profit_factor
ic_ir_return,54.7,1.84,1.54,-25.6,-26.84,57.13,62.45,76.56,100.0,1.42,-1.45,0.22,0.97,1.37


In [35]:
bt_ret_last10days = bt_ret.iloc[-10:] * 100
bt_ret_last10days.iloc[0] = 0
bt_ret_last10days.cumsum().iplot(title='累積報酬 % ')

In [36]:
績效回顧 = bt_ret.reindex(exp_ret.index).shift(2)
display((績效回顧.iloc[-21:]*100).round(2))

,ic_ir_return
mdate,
2025-03-19,-0.35
2025-03-20,-0.13
2025-03-21,0.49
2025-03-24,-0.25
2025-03-25,-0.72
2025-03-26,0.20
2025-03-27,0.11
2025-03-28,-1.83
2025-03-31,-3.71


In [37]:
def check_roll_contracts_for_weekly_strategy(weight_df, top_contracts, target_date="2023-03-31"):
    import pandas as pd

    # 轉換成 datetime
    target_date = pd.to_datetime(target_date)

    # 找到該週的最後一個交易日
    week_dates = weight_df.index.to_series().dt.to_period("W")
    week_last_day = weight_df[week_dates == target_date.to_period("W")].index.max()

    if pd.isna(week_last_day):
        print(f"找不到 {target_date.strftime('%Y-%m-%d')} 所在週的最後交易日")
        return

    # 找下一個交易日作為進場日
    all_dates = weight_df.index.sort_values()
    next_day_arr = all_dates[all_dates > week_last_day]
    if len(next_day_arr) == 0:
        print(f"{week_last_day.date()} 之後沒有交易日")
        return

    signal_date = week_last_day
    next_day = next_day_arr[0]

    print(f"\n週末訊號日：{signal_date.date()}，進場日：{next_day.date()}")

    # 當週持股
    held = weight_df.loc[signal_date]
    held = held[held > 0]

    # 查期貨合約
    for coid in held.index:
        match = top_contracts[
            (top_contracts['年月日'] == next_day) &
            (top_contracts['標的證券'].astype(str).str.contains(str(coid)))
        ]
        if match.empty:
            print(f"{coid}：{next_day.date()} 無期貨資料")
        else:
            contract = match.iloc[0]['證券代碼']
            print(f"{coid} → 合約：{contract}")

In [59]:
check_roll_contracts_for_weekly_strategy(weightings['ic_ir'], top_contracts, target_date="2025-04-13")


週末訊號日：2025-04-11，進場日：2025-04-14
2317 → 合約：DHF202504 鴻海期貨 2025/04
2330 → 合約：QFF202504 小型台積電 2025/04
2382 → 合約：DKF202504 廣達期貨 2025/04
2454 → 合約：PUF202504 小型聯發期 2025/04
2603 → 合約：CZF202504 長榮期貨 2025/04
2634 → 合約：RCF202504 漢翔期貨 2025/04
3231 → 合約：DXF202504 緯創期貨 2025/04
3481 → 合約：DQF202504 群創期貨 2025/04
5274 → 合約：QAF202504 小型信驊 2025/04
6669 → 合約：PWF202504 小型緯穎 2025/04


In [62]:
def get_contracts_from_weights(weight_df, top_contracts, target_date=None):
    import pandas as pd

    # 確保 top_contracts 中日期格式正確
    top_contracts = top_contracts.copy()
    top_contracts['年月日'] = pd.to_datetime(top_contracts['年月日'])

    result = []

    # 若有指定日期，轉成 list；否則用全部日期
    if target_date is not None:
        dates_to_check = [pd.to_datetime(target_date)]
    else:
        dates_to_check = weight_df.index

    for dt in dates_to_check:
        if dt not in weight_df.index:
            print(f"❌ {dt.date()} 不在 weight_df 中")
            continue

        held = weight_df.loc[dt]
        held = held[held > 0]

        if held.empty:
            print(f"⚠️ {dt.date()} 無持倉")
            continue

        for coid, wt in held.items():
            match = top_contracts[
                (top_contracts['年月日'] == dt) &
                (top_contracts['標的證券'].astype(str).str.contains(str(coid)))
            ]
            if match.empty:
                contract = None
            else:
                contract = match.iloc[0]['證券代碼']
            result.append({
                "日期": dt,
                "股票代碼": coid,
                "權重": wt,
                "期貨合約": contract
            })

    return pd.DataFrame(result)


In [101]:
get_contracts_from_weights(weightings['ic_ir'], top_contracts, target_date="2025-04-18")

,日期,股票代碼,權重,期貨合約
0,2025-04-18,2317,0.1,DHF202505 鴻海期貨 2025/05
1,2025-04-18,2330,0.1,QFF202505 小型台積電 2025/05
2,2025-04-18,2382,0.1,DKF202505 廣達期貨 2025/05
3,2025-04-18,2454,0.1,PUF202505 小型聯發期 2025/05
4,2025-04-18,2603,0.1,CZF202505 長榮期貨 2025/05
5,2025-04-18,2634,0.1,RCF202505 漢翔期貨 2025/05
6,2025-04-18,3231,0.1,DXF202505 緯創期貨 2025/05
7,2025-04-18,3481,0.1,DQF202505 群創期貨 2025/05
8,2025-04-18,5274,0.1,QAF202505 小型信驊 2025/05
9,2025-04-18,6669,0.1,PWF202505 小型緯穎 2025/05
